# Production-Ready Infrastructure

## Полный Production Setup с Best Practices

Этот туториал демонстрирует создание production-ready инфраструктуры со всеми необходимыми компонентами:

- ✅ **High Availability**: Multi-AZ deployment
- ✅ **Security**: WAF, Secrets Management, Network Isolation
- ✅ **Monitoring**: Comprehensive observability stack
- ✅ **Backup & DR**: Automated backups, disaster recovery
- ✅ **CI/CD**: Automated deployment pipeline
- ✅ **Cost Optimization**: Auto-scaling, spot instances
- ✅ **Compliance**: Logging, auditing, encryption

### Production Architecture

```
┌─────────────────────────────────────────────────┐
│  Route53 (DNS) + Health Checks                  │
└──────────────┬──────────────────────────────────┘
               │
┌──────────────▼──────────────────────────────────┐
│  CloudFront (CDN) + WAF                         │
└──────────────┬──────────────────────────────────┘
               │
┌──────────────▼──────────────────────────────────┐
│  ALB (Multi-AZ) + SSL/TLS                       │
└──────┬───────────────────────┬──────────────────┘
       │                       │
┌──────▼─────┐         ┌───────▼──────┐
│  AZ-1      │         │    AZ-2      │
│            │         │              │
│ ECS Tasks  │         │  ECS Tasks   │
│ (Fargate)  │         │  (Fargate)   │
└──────┬─────┘         └───────┬──────┘
       │                       │
       └───────────┬───────────┘
                   │
       ┌───────────▼───────────┐
       │  RDS Multi-AZ         │
       │  + Read Replicas      │
       └───────────────────────┘
```

## Шаг 1: Базовая инфраструктура с HA

In [ ]:
%%writefile production/main.tf

terraform {
  required_version = ">= 1.5.0"
  
  backend "s3" {
    bucket         = "production-terraform-state"
    key            = "production/terraform.tfstate"
    region         = "us-east-1"
    encrypt        = true
    dynamodb_table = "terraform-state-lock"
    kms_key_id     = "arn:aws:kms:us-east-1:ACCOUNT:key/KEY_ID"
  }
}

locals {
  environment = "production"
  project     = "myapp"
  
  common_tags = {
    Environment = local.environment
    Project     = local.project
    ManagedBy   = "Terraform"
    CostCenter  = "Engineering"
  }
}

# Multi-AZ VPC
module "vpc" {
  source = "git::https://github.com/v-grand/infra-network.git//modules/vpc"

  project_name = local.project
  environment  = local.environment
  vpc_cidr     = "10.0.0.0/16"

  # 3 AZs for high availability
  azs = [
    "us-east-1a",
    "us-east-1b",
    "us-east-1c"
  ]
  
  private_subnets = [
    "10.0.1.0/24",
    "10.0.2.0/24",
    "10.0.3.0/24"
  ]
  
  public_subnets = [
    "10.0.101.0/24",
    "10.0.102.0/24",
    "10.0.103.0/24"
  ]
  
  database_subnets = [
    "10.0.201.0/24",
    "10.0.202.0/24",
    "10.0.203.0/24"
  ]

  enable_nat_gateway   = true
  single_nat_gateway   = false  # NAT in each AZ for HA
  enable_dns_hostnames = true
  enable_dns_support   = true
  
  # VPC Flow Logs for security
  enable_flow_log                      = true
  create_flow_log_cloudwatch_iam_role  = true
  create_flow_log_cloudwatch_log_group = true

  tags = local.common_tags
}

## Шаг 2: Secrets Management с SOPS и AWS Secrets Manager

In [ ]:
%%writefile production/secrets.tf

# KMS Key for encryption
resource "aws_kms_key" "secrets" {
  description             = "KMS key for secrets encryption"
  deletion_window_in_days = 30
  enable_key_rotation     = true

  tags = merge(local.common_tags, {
    Name = "${local.project}-secrets-key"
  })
}

resource "aws_kms_alias" "secrets" {
  name          = "alias/${local.project}-secrets"
  target_key_id = aws_kms_key.secrets.key_id
}

# Database credentials
resource "aws_secretsmanager_secret" "db_credentials" {
  name        = "${local.project}/database/credentials"
  kms_key_id  = aws_kms_key.secrets.id
  
  recovery_window_in_days = 7
  
  tags = local.common_tags
}

resource "aws_secretsmanager_secret_version" "db_credentials" {
  secret_id = aws_secretsmanager_secret.db_credentials.id
  secret_string = jsonencode({
    username = "admin"
    password = random_password.db_password.result
    engine   = "postgres"
    host     = module.rds.db_instance_address
    port     = 5432
    dbname   = "production_db"
  })
}

resource "random_password" "db_password" {
  length  = 32
  special = true
}

# API Keys
resource "aws_secretsmanager_secret" "api_keys" {
  name       = "${local.project}/api/keys"
  kms_key_id = aws_kms_key.secrets.id
  
  tags = local.common_tags
}

# JWT Secret
resource "aws_secretsmanager_secret" "jwt_secret" {
  name       = "${local.project}/jwt/secret"
  kms_key_id = aws_kms_key.secrets.id
  
  tags = local.common_tags
}

In [ ]:
# Создание SOPS конфигурации
!cat > .sops.yaml << 'EOF'
creation_rules:
  - path_regex: secrets/production/.*\.yaml$
    kms: 'arn:aws:kms:us-east-1:ACCOUNT_ID:key/KEY_ID'
    aws_profile: production
  
  - path_regex: secrets/staging/.*\.yaml$
    kms: 'arn:aws:kms:us-east-1:ACCOUNT_ID:key/STAGING_KEY_ID'
    aws_profile: staging
EOF

print("✅ SOPS configuration created")

## Шаг 3: High-Availability Database (RDS Multi-AZ)

In [ ]:
%%writefile production/database.tf

module "rds" {
  source = "git::https://github.com/v-grand/infra-aws.git//modules/rds"

  identifier = "${local.project}-production"
  
  engine               = "postgres"
  engine_version       = "15.4"
  family               = "postgres15"
  major_engine_version = "15"
  instance_class       = "db.r6g.xlarge"

  allocated_storage     = 100
  max_allocated_storage = 1000
  storage_encrypted     = true
  kms_key_id           = aws_kms_key.secrets.arn

  db_name  = "production_db"
  username = "admin"
  password = random_password.db_password.result
  port     = 5432

  # High Availability
  multi_az               = true
  db_subnet_group_name   = module.vpc.database_subnet_group_name
  vpc_security_group_ids = [aws_security_group.rds.id]

  # Backups
  backup_retention_period = 30
  backup_window          = "03:00-04:00"
  maintenance_window     = "Mon:04:00-Mon:05:00"
  
  # Enhanced Monitoring
  enabled_cloudwatch_logs_exports = ["postgresql", "upgrade"]
  create_monitoring_role          = true
  monitoring_interval             = 60
  monitoring_role_name            = "${local.project}-rds-monitoring"

  # Performance Insights
  performance_insights_enabled    = true
  performance_insights_kms_key_id = aws_kms_key.secrets.arn
  
  # Deletion protection
  deletion_protection = true
  skip_final_snapshot = false
  final_snapshot_identifier_prefix = "${local.project}-final"

  tags = local.common_tags
}

# Read Replica for read-heavy workloads
resource "aws_db_instance" "read_replica" {
  identifier             = "${local.project}-read-replica"
  replicate_source_db    = module.rds.db_instance_id
  instance_class         = "db.r6g.large"
  
  publicly_accessible    = false
  skip_final_snapshot    = true
  
  # Performance Insights
  performance_insights_enabled = true
  
  tags = merge(local.common_tags, {
    Name = "${local.project}-read-replica"
  })
}

## Шаг 4: Application Layer (ECS Fargate с Auto-Scaling)

In [ ]:
%%writefile production/ecs.tf

module "ecs" {
  source = "git::https://github.com/v-grand/infra-aws.git//modules/ecs"

  cluster_name = "${local.project}-production"
  
  vpc_id          = module.vpc.vpc_id
  private_subnets = module.vpc.private_subnets
  public_subnets  = module.vpc.public_subnets

  # Container Insights for monitoring
  container_insights = true

  services = {
    api = {
      cpu                      = 1024
      memory                   = 2048
      desired_count            = 3
      image                    = "${var.ecr_registry}/api:latest"
      port                     = 8000
      health_check_path        = "/health"
      health_check_grace_period = 60
      
      # Auto-scaling
      enable_autoscaling = true
      min_capacity       = 3
      max_capacity       = 20
      
      autoscaling_policies = {
        cpu = {
          target_value       = 70
          scale_in_cooldown  = 300
          scale_out_cooldown = 60
        }
        memory = {
          target_value       = 80
          scale_in_cooldown  = 300
          scale_out_cooldown = 60
        }
        requests = {
          target_value       = 1000
          scale_in_cooldown  = 300
          scale_out_cooldown = 60
        }
      }
      
      # Secrets from AWS Secrets Manager
      secrets = [
        {
          name      = "DB_CREDENTIALS"
          valueFrom = aws_secretsmanager_secret.db_credentials.arn
        },
        {
          name      = "JWT_SECRET"
          valueFrom = aws_secretsmanager_secret.jwt_secret.arn
        }
      ]
      
      # Environment variables
      environment = [
        {
          name  = "ENVIRONMENT"
          value = "production"
        },
        {
          name  = "LOG_LEVEL"
          value = "INFO"
        },
        {
          name  = "DB_READ_REPLICA"
          value = aws_db_instance.read_replica.address
        }
      ]
    }
  }

  tags = local.common_tags
}

## Шаг 5: Security (WAF, Security Groups, NACLs)

In [ ]:
%%writefile production/security.tf

# WAF for CloudFront and ALB
resource "aws_wafv2_web_acl" "main" {
  name  = "${local.project}-waf"
  scope = "REGIONAL"

  default_action {
    allow {}
  }

  # Rate limiting
  rule {
    name     = "rate-limit"
    priority = 1

    action {
      block {}
    }

    statement {
      rate_based_statement {
        limit              = 2000
        aggregate_key_type = "IP"
      }
    }

    visibility_config {
      cloudwatch_metrics_enabled = true
      metric_name                = "RateLimit"
      sampled_requests_enabled   = true
    }
  }

  # AWS Managed Rules - Core Rule Set
  rule {
    name     = "aws-managed-core-rules"
    priority = 2

    override_action {
      none {}
    }

    statement {
      managed_rule_group_statement {
        name        = "AWSManagedRulesCommonRuleSet"
        vendor_name = "AWS"
      }
    }

    visibility_config {
      cloudwatch_metrics_enabled = true
      metric_name                = "AWSManagedRulesCommonRuleSet"
      sampled_requests_enabled   = true
    }
  }

  # SQL Injection protection
  rule {
    name     = "sql-injection-protection"
    priority = 3

    override_action {
      none {}
    }

    statement {
      managed_rule_group_statement {
        name        = "AWSManagedRulesSQLiRuleSet"
        vendor_name = "AWS"
      }
    }

    visibility_config {
      cloudwatch_metrics_enabled = true
      metric_name                = "SQLInjectionProtection"
      sampled_requests_enabled   = true
    }
  }

  visibility_config {
    cloudwatch_metrics_enabled = true
    metric_name                = "${local.project}-waf"
    sampled_requests_enabled   = true
  }

  tags = local.common_tags
}

# Security Group for ALB
resource "aws_security_group" "alb" {
  name_prefix = "${local.project}-alb-"
  vpc_id      = module.vpc.vpc_id

  ingress {
    from_port   = 443
    to_port     = 443
    protocol    = "tcp"
    cidr_blocks = ["0.0.0.0/0"]
  }

  ingress {
    from_port   = 80
    to_port     = 80
    protocol    = "tcp"
    cidr_blocks = ["0.0.0.0/0"]
  }

  egress {
    from_port   = 0
    to_port     = 0
    protocol    = "-1"
    cidr_blocks = ["0.0.0.0/0"]
  }

  tags = merge(local.common_tags, {
    Name = "${local.project}-alb-sg"
  })
}

# Security Group for RDS
resource "aws_security_group" "rds" {
  name_prefix = "${local.project}-rds-"
  vpc_id      = module.vpc.vpc_id

  ingress {
    from_port       = 5432
    to_port         = 5432
    protocol        = "tcp"
    security_groups = [module.ecs.service_security_group_id]
  }

  tags = merge(local.common_tags, {
    Name = "${local.project}-rds-sg"
  })
}

## Шаг 6: Comprehensive Monitoring Stack

In [ ]:
%%writefile production/monitoring.tf

# CloudWatch Dashboard
resource "aws_cloudwatch_dashboard" "main" {
  dashboard_name = "${local.project}-production"

  dashboard_body = jsonencode({
    widgets = [
      {
        type = "metric"
        properties = {
          metrics = [
            ["AWS/ECS", "CPUUtilization", { stat = "Average" }],
            [".", "MemoryUtilization", { stat = "Average" }]
          ]
          period = 300
          stat   = "Average"
          region = var.aws_region
          title  = "ECS Metrics"
        }
      },
      {
        type = "metric"
        properties = {
          metrics = [
            ["AWS/RDS", "CPUUtilization"],
            [".", "DatabaseConnections"],
            [".", "FreeableMemory"]
          ]
          period = 300
          stat   = "Average"
          region = var.aws_region
          title  = "RDS Metrics"
        }
      }
    ]
  })
}

# SNS Topic for alerts
resource "aws_sns_topic" "alerts" {
  name = "${local.project}-alerts"
  
  tags = local.common_tags
}

# CloudWatch Alarms
resource "aws_cloudwatch_metric_alarm" "ecs_cpu_high" {
  alarm_name          = "${local.project}-ecs-cpu-high"
  comparison_operator = "GreaterThanThreshold"
  evaluation_periods  = "2"
  metric_name         = "CPUUtilization"
  namespace           = "AWS/ECS"
  period              = "300"
  statistic           = "Average"
  threshold           = "80"
  alarm_description   = "ECS CPU utilization is too high"
  alarm_actions       = [aws_sns_topic.alerts.arn]
}

resource "aws_cloudwatch_metric_alarm" "rds_cpu_high" {
  alarm_name          = "${local.project}-rds-cpu-high"
  comparison_operator = "GreaterThanThreshold"
  evaluation_periods  = "2"
  metric_name         = "CPUUtilization"
  namespace           = "AWS/RDS"
  period              = "300"
  statistic           = "Average"
  threshold           = "80"
  alarm_description   = "RDS CPU utilization is too high"
  alarm_actions       = [aws_sns_topic.alerts.arn]
  
  dimensions = {
    DBInstanceIdentifier = module.rds.db_instance_id
  }
}

## Шаг 7: Backup & Disaster Recovery

In [ ]:
%%writefile production/backup.tf

# AWS Backup Vault
resource "aws_backup_vault" "main" {
  name        = "${local.project}-backup-vault"
  kms_key_arn = aws_kms_key.secrets.arn
  
  tags = local.common_tags
}

# Backup Plan
resource "aws_backup_plan" "main" {
  name = "${local.project}-backup-plan"

  # Daily backups
  rule {
    rule_name         = "daily_backup"
    target_vault_name = aws_backup_vault.main.name
    schedule          = "cron(0 2 * * ? *)"

    lifecycle {
      delete_after = 30
    }
  }

  # Weekly backups
  rule {
    rule_name         = "weekly_backup"
    target_vault_name = aws_backup_vault.main.name
    schedule          = "cron(0 3 ? * 1 *)"

    lifecycle {
      delete_after       = 90
      cold_storage_after = 30
    }
  }

  # Monthly backups
  rule {
    rule_name         = "monthly_backup"
    target_vault_name = aws_backup_vault.main.name
    schedule          = "cron(0 4 1 * ? *)"

    lifecycle {
      delete_after       = 365
      cold_storage_after = 90
    }
  }

  tags = local.common_tags
}

# Backup Selection
resource "aws_backup_selection" "main" {
  name         = "${local.project}-backup-selection"
  plan_id      = aws_backup_plan.main.id
  iam_role_arn = aws_iam_role.backup.arn

  resources = [
    module.rds.db_instance_arn,
    # Add other resources to backup
  ]
  
  selection_tag {
    type  = "STRINGEQUALS"
    key   = "Backup"
    value = "true"
  }
}

## Шаг 8: CI/CD Pipeline

In [ ]:
%%writefile .github/workflows/production-deploy.yml

name: Production Deployment

on:
  push:
    branches: [main]
    tags: ['v*']

env:
  AWS_REGION: us-east-1
  ECR_REPOSITORY: myapp-api
  ECS_CLUSTER: myapp-production
  ECS_SERVICE: api

jobs:
  security-scan:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      
      - name: Run Trivy vulnerability scanner
        uses: aquasecurity/trivy-action@master
        with:
          scan-type: 'fs'
          scan-ref: '.'
          severity: 'CRITICAL,HIGH'

  terraform:
    needs: security-scan
    uses: v-grand/infra-ci/.github/workflows/terraform-plan.yml@main
    with:
      terraform_version: '1.5.0'
      working_directory: './production'
    secrets: inherit

  build-and-push:
    needs: terraform
    runs-on: ubuntu-latest
    outputs:
      image: ${{ steps.build-image.outputs.image }}
    
    steps:
      - uses: actions/checkout@v3
      
      - name: Configure AWS credentials
        uses: aws-actions/configure-aws-credentials@v2
        with:
          aws-access-key-id: ${{ secrets.AWS_ACCESS_KEY_ID }}
          aws-secret-access-key: ${{ secrets.AWS_SECRET_ACCESS_KEY }}
          aws-region: ${{ env.AWS_REGION }}
      
      - name: Login to Amazon ECR
        id: login-ecr
        uses: aws-actions/amazon-ecr-login@v1
      
      - name: Build, tag, and push image
        id: build-image
        env:
          ECR_REGISTRY: ${{ steps.login-ecr.outputs.registry }}
          IMAGE_TAG: ${{ github.sha }}
        run: |
          docker build -t $ECR_REGISTRY/$ECR_REPOSITORY:$IMAGE_TAG .
          docker push $ECR_REGISTRY/$ECR_REPOSITORY:$IMAGE_TAG
          echo "image=$ECR_REGISTRY/$ECR_REPOSITORY:$IMAGE_TAG" >> $GITHUB_OUTPUT

  deploy:
    needs: build-and-push
    runs-on: ubuntu-latest
    environment: production
    
    steps:
      - name: Deploy to ECS
        run: |
          aws ecs update-service \
            --cluster $ECS_CLUSTER \
            --service $ECS_SERVICE \
            --force-new-deployment \
            --region $AWS_REGION
      
      - name: Wait for deployment
        run: |
          aws ecs wait services-stable \
            --cluster $ECS_CLUSTER \
            --services $ECS_SERVICE \
            --region $AWS_REGION
      
      - name: Notify Slack
        uses: 8398a7/action-slack@v3
        with:
          status: ${{ job.status }}
          text: 'Production deployment completed'
        env:
          SLACK_WEBHOOK_URL: ${{ secrets.SLACK_WEBHOOK }}

## Проверка Production Infrastructure

In [ ]:
import boto3
import json

def check_production_health():
    """Comprehensive production health check"""
    
    # ECS Health
    ecs = boto3.client('ecs', region_name='us-east-1')
    services = ecs.describe_services(
        cluster='myapp-production',
        services=['api']
    )
    
    print("=== ECS Service Health ===")
    for svc in services['services']:
        print(f"Service: {svc['serviceName']}")
        print(f"  Status: {svc['status']}")
        print(f"  Running: {svc['runningCount']}/{svc['desiredCount']}")
        print(f"  Deployment: {svc['deployments'][0]['status']}")
    
    # RDS Health
    rds = boto3.client('rds', region_name='us-east-1')
    db = rds.describe_db_instances(
        DBInstanceIdentifier='myapp-production'
    )
    
    print("\n=== RDS Health ===")
    for instance in db['DBInstances']:
        print(f"Database: {instance['DBInstanceIdentifier']}")
        print(f"  Status: {instance['DBInstanceStatus']}")
        print(f"  Multi-AZ: {instance['MultiAZ']}")
        print(f"  Backup Retention: {instance['BackupRetentionPeriod']} days")
    
    # CloudWatch Alarms
    cw = boto3.client('cloudwatch', region_name='us-east-1')
    alarms = cw.describe_alarms(
        AlarmNamePrefix='myapp-'
    )
    
    print("\n=== CloudWatch Alarms ===")
    for alarm in alarms['MetricAlarms']:
        print(f"{alarm['AlarmName']}: {alarm['StateValue']}")
    
    # WAF Status
    waf = boto3.client('wafv2', region_name='us-east-1')
    web_acls = waf.list_web_acls(Scope='REGIONAL')
    
    print("\n=== WAF Status ===")
    for acl in web_acls['WebACLs']:
        print(f"WAF ACL: {acl['Name']} - {acl['ARN']}")

check_production_health()

## Заключение

### Production-Ready Infrastructure включает:

✅ **High Availability**: Multi-AZ deployment, RDS Multi-AZ, Read Replicas  
✅ **Security**: WAF, Security Groups, KMS encryption, Secrets Manager  
✅ **Monitoring**: CloudWatch Dashboards, Alarms, Container Insights  
✅ **Backup & DR**: AWS Backup with daily/weekly/monthly retention  
✅ **Auto-Scaling**: ECS Service auto-scaling based on CPU/Memory/Requests  
✅ **CI/CD**: Automated deployment with security scanning  
✅ **Compliance**: VPC Flow Logs, CloudTrail, encryption at rest and in transit  
✅ **Cost Optimization**: Auto-scaling, Spot instances support  

### Production Checklist:

- [ ] SSL/TLS certificates configured (ACM)
- [ ] DNS configured (Route53)
- [ ] Monitoring alerts configured (SNS)
- [ ] Backup tested and verified
- [ ] Disaster recovery plan documented
- [ ] Security audit completed
- [ ] Load testing performed
- [ ] Runbook created for operations team
- [ ] Cost alerts configured
- [ ] Compliance requirements met

### Operational Commands:

```bash
# Deploy infrastructure
cd production && terraform apply

# View logs
aws logs tail /ecs/myapp-production --follow

# Manual scaling
aws ecs update-service --cluster myapp-production --service api --desired-count 10

# Database backup
aws rds create-db-snapshot --db-instance-identifier myapp-production --db-snapshot-identifier manual-backup-$(date +%Y%m%d)

# Rollback deployment
aws ecs update-service --cluster myapp-production --service api --task-definition myapp-api:PREVIOUS_VERSION
```